In [ ]:
pip install pymystem3

In [ ]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem #импорт класса лемматизатора pymystem3
#import pymorphy2 #импорт инструмента лемматизатора pymorphy2
import re #импорт модуля для работы с регулярными выражениями (для очистки)

import nltk #импорт библиотеки для работы с Natural Language
from nltk.corpus import stopwords as nltk_stopwords #импорт инструмента для определения стоп-слов
nltk.download('stopwords') #скачиваем стоп-слова
from string import punctuation

from tqdm import notebook #импорт инструмента процесса выполнения циклов

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #импорт инструмента для расчёта TF-IDF
from sklearn.metrics import f1_score, make_scorer,accuracy_score #импорт метрики f1, инструмента для интеграции метрики в gridsearch
from scipy.spatial import distance #измерение косинусного расстояния
from sklearn.metrics.pairwise import cosine_similarity #измерение косинусной близости векторов
from sklearn.cluster import KMeans #импорт модели обучения без учителя методомм k-средних

from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV #импорт инструментов выбора модели
from sklearn.ensemble import RandomForestClassifier #импорт модели случайного леса
from sklearn.linear_model import LogisticRegression #импорт модели логистической регрессии
from sklearn.pipeline import Pipeline

from google.colab import drive #импорт инструмента для работы с файлами на google drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pd.__version__

'1.5.3'

In [ ]:
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', 200)# Установите для отображения самой большой линии

# Загрузка данных


In [ ]:
# подключаем google drive
drive.mount('/content/drive')
#загружаем файл
df_2 = pd.read_excel('./drive/MyDrive/Colab Notebooks/Хакатоны/posts.xlsx',nrows=10000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Решение задачи №1

Необходимо создать алгоритм, который способен точно и быстро идентифицировать пары текстов, содержащих схожие или практически идентичные новости.

Сравнивать тексты предполагается следующим образом:
* чистим тексты
* лемматизируем тексты
* векторизуем тексты при помощи tf-idf
* сравниваем вектора между собой

Чем ближе вектора - тем более они похожи, из схожих выбираем самые короткие и удаляем их

In [ ]:
#сохраним df_2 в основной датасет для работы - df
df=df_2
#оставим только столбец 'text'
df=df['text']
#удаление дубликатов
df=df.drop_duplicates()
#удаление пропусков
df=df.dropna()
#обновление индексов после всех манипуляций
df=df.reset_index(drop=True)

# Решение для заголовков TF-IDF

Итак, в итоге получаем датасет с уникальными заголовками и текстами новостей, очищенный от дубликатов, пропусков и с обновлёнными индексами

In [ ]:
#создадим функцию, которая оставит в тексте только русскоязычные буквы
def clear_text(text):
    text_0=re.sub(r'[^а-яА-ЯёЁ]',' ',text)
    text_1=text_0.split()
    text_2=" ".join(text_1)
    return text_2.lower()

#создание функции для лемматизации
m = Mystem()
def lemmatize(text):
    lemm_text=m.lemmatize(text)
    return "".join(lemm_text)

#сохранение русских стоп-слов
stopwords=set(nltk_stopwords.words('russian'))
#обозначение инструмента подсчёта tf-idf
count_tf_idf=TfidfVectorizer(stop_words=list(stopwords))

#функция подсчёта схожести для двух текстов
#text - датафрейм целиком, column - колнка, по которой сравниваем тексты (заголовок или текст)
#batch_size - размер порции для расчёта tf-idf (по умолчанию 3000)
#threshold - порог схожести текстов (от 0 до 1): чем больше, тем они должны быть более похожи пословно
#(по умолчанию 0.32, подобран самостоятельно на основе исходного датасета)
def same_text_c(text,batch_size=3000,threshold=0.32):
    corpus=text.copy()
    corpus=corpus.astype('str')
    #очистка текстов
    for i in notebook.tqdm(range(len(corpus))):
        corpus[i]=clear_text(corpus[i])

    #лемматизация текстов
    for i in notebook.tqdm(range(len(corpus))):
        corpus[i]=lemmatize(corpus[i])

    #формируем порции векторов
    batch_size=batch_size
    for batch in notebook.tqdm(range(round(len(corpus)/batch_size))):
      corpus_part=corpus[batch_size*batch:batch_size*(batch+1)]

      #подсчёт TF-IDF для каждой порции
      tf_idf=count_tf_idf.fit_transform(corpus_part)
      cs=cosine_similarity(tf_idf)

      #сохраняем список для всех отобранных новостей, которые короче
      d2=[]
      #проходим по элементам массива cs
      #сравниваем между соседними 600 строками (это примерно 1 день по датасету):
      #повторяющаяся новость за следующий день как правило уже серьёзно обновлена
      for i in range(len(cs)-600):
        for j in range(i+1,i+600):
          dist=cs[i][j]
          if dist>threshold:
            if len(corpus[j+batch_size*batch])<len(corpus[i+batch_size*batch]):
              d2.append(j+batch_size*batch)
            else:
              d2.append(i+batch_size*batch)
      #удаляем короткие похожие строки
      text=text.drop(set(d2))
    #вывод функции
    return text

In [ ]:
%%time
d=same_text_c(df)
d

  0%|          | 0/9887 [00:00<?, ?it/s]

  0%|          | 0/9887 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 9.45 s, sys: 412 ms, total: 9.86 s
Wall time: 41 s


0       ❗️Восстановление аммиакопровода Тольятти — Одесса займет от 1 до 3 месяцев при наличии доступа к объекту, заявили в МИД РФ.\n\nВ ведомстве отметили, что Россия предпримет усилия для выяснения обст...
1       Дополнительные 39 миллионов рублей были выделены на социальные учреждения Марий Эл. Об этом заявили участники Съезда соцработников.\n\nСемь автобусов для детских учреждений будут закуплены на 29 м...
2                                                                                              «Россия уничтожена санкциями»\n                          Байден.\nАх-ха-ха:)))\n\nhttps://t.me/rt_russian/160646
3       ❗️Правоохранители провели обыски в министерстве образования и науки Дагестана, сообщает ТАСС. \n\nВ правоохранительных органах уточнили, что обыски прошли по уголовному делу, связанному с выплатам...
5       👧👦 В загородных детских лагерях Марий Эл в данный момент отдыхают 2335 детей, и ещё 12265 ребят посещают 175 пришкольных лагерей.\n\nВсего на территории республ

## Разметка (метод k-средних)

Теперь задача категоризировать очищенный текст по заранее известным 29 категориям.

Реализуем следующую идею:
* категории опишем словами-тегами
* векторизуем тексты и слова-теги единым векторизатором
* сначала категорзируем без учителя на небольшой выборке методом k-средних на 29 категорий: получаем разметку 1000 строк
* на этих 1000 строках обучаем модель с учителем, и уже её применяем для всего датасета

Почему применяем вторую модель, а не категоризируем всё моделью без учителя? Ответ: дело в том, что данной стратегией (когда мы задали теги для категорий), в какой-то момент группы, по которым алогритм k-средних делит все тексты, увеличиваются и перекрывают друг друга, тем самым вызывая переобучение и уменьшение количества категорий.
В принципе, если бы категорий было значительно меньше и если бы они сильно отличались друг от друга, то это могло бы и сработать.

Дополнительная модель позволяет взять все категории, которые есть, и учиться на них.


In [ ]:
%%time
#выборка текстов
n1=0
n2=1000
d=d.reset_index(drop=True)
#выборка обрабатываемых строк
corpus=d[n1:n2].copy()
#выборка строк, которые пойдут в финал
text=pd.DataFrame(d[n1:n2].copy())
#обозначение кластеров
clusters=['Блоги','Новости и СМИ','Развлечения и юмор','Технологии','Экономика','Бизнес и стартапы','Криптовалюты',
          'Путешествия','Маркетинг, PR, реклама','Психология','Дизайн','Политика','Искусство','Право',
          'Образование и познавательное','Спорт','Мода и красота','Здоровье и медицина','Картинки и фото',
          'Софт и приложения','Видео и фильмы','Музыка','Игры','Еда и кулинария','Цитаты','Рукоделие', 'Финансы',
          'Шоубиз','Другое (раньше "общее")']
#сочинение на тему подбора синонимов для названий кластеров
clusters_tags=['ресурс видеоблог телеграм выкладывать военкор выложить инфлюенсер вконтакте вк инстаграм блогер блог контент подписичик',
               'новость телевизор тасс россия 24 канал первый лента новости сми агенство репортаж репортер ',
               'Развлечения юмор шутка смех мем прикол',
               'Технологии айфон телефон айти инженер нанотехнология биотехнология интернет',
               'Экономика рецессия банк ставка ключевая облигации активы инфляция центробанк ЦБ ',
               'Бизнес и стартапы компания технология инвестиция ',
               'Криптовалюты биткоин ключ майнить ферма видеокарта usdt шифрование ',
               'Путешествия граница виза въезд пребывание авиасообщение авиабилеты отпуск',
               'Маркетинг, PR, реклама продажи  ',
               'Психология психолог терапия психотерапия дети секс отношения муж жена семья ',
               'Дизайн разработка оформление архитектура ландшафт цвет шрифт размер',
               'Политика МИД РФ операция СВО военная путин президент зеленский сша россия украина евросоюз ',
               'Искусство выставка музей картина  опера ',
               'Право судья адвокат подозреваемый прокурор преступление закон законодательство полномочия лицензия иммунитет юрист юриспруденция ',
               'Образование школа гимназия обучение университет институт диплом ',
               'Спорт футбол соревнование стадион волейбол баскетбол хоккей счёт судья спортсмен олимпиада флаг коньки',
               'Мода и красота одежда магазин торговый бренд модель писк сезон стрижка косметика',
               'Здоровье и медицина ковид коронавирус штамм лечение страхование поликилника врач протез анализ медики препарат',
               'Картинки и фото изображение показать фотография фотограф запечатлеть',
               'Софт и приложения компьютер разработчик рустор телефон андроид аврора эпл',
               'Видео и фильмы кино кинотеатр сериал мультфильм триллер',
               'Музыка концерт зал клуб рок блюз петь сцена фанаты песня выступать поп-музыка вокал музыка музыка музыкант гитарист вокалист нейромузыка фронтмен',
               'Игры настольный мобильный теннис олимпийский шахматы',
               'Еда и кулинария ресторан кафе фастфуд вкусно ростикс перекус кофейня сеть',
               'Цитаты сказал по мнению ответил объявил цитировать объявление объявил сказать заявление',
               'Рукоделие дело ручная работа труд занятие ремесло вышивка',
               'Финансы деньги рубль доллар евро акции торги ',
               'Шоубиз звезда актёр актриса театр  скандал ',
               'Другое общее всякое разное природа климат  ']
#добавляем теги кластеров в конец выборки
corpus=corpus.append(pd.Series(clusters_tags))
corpus=corpus.values.astype('U')

#очистка текстов
for i in notebook.tqdm(range(len(corpus))):
  corpus[i]=clear_text(corpus[i])

#лемматизация текстов
for i in notebook.tqdm(range(len(corpus))):
  corpus[i]=lemmatize(corpus[i])

nltk.download('stopwords')

#сохранение русских стоп-слов
stopwords=set(nltk_stopwords.words('russian'))
#обозначение инструмента подсчёта tf-idf
count_tf_idf=TfidfVectorizer(stop_words=list(stopwords))

#подсчёт TF-IDF
tf_idf=count_tf_idf.fit_transform(corpus)
#забираем последние строки и используем их вектора как центроиды для дальнейшего обучения
centers=tf_idf[len(text):len(text)+len(clusters_tags)].toarray()
#обучем модель
model=KMeans(n_clusters=len(clusters),init=centers,random_state=12345).fit(tf_idf)
#выводим результат категоризации последних строк (проверка)
model.labels_[len(text):len(text)+len(clusters_tags)]

<timed exec>:46: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


  0%|          | 0/1029 [00:00<?, ?it/s]

  0%|          | 0/1029 [00:00<?, ?it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1362: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)


CPU times: user 815 ms, sys: 49.2 ms, total: 865 ms
Wall time: 3.73 s


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23,  3, 25, 26, 27, 28], dtype=int32)

В идеальном мире мы бы получили соответствие каждой категории своему номеру: от 0 до 28, но центроиды-вектора оказались не слишком далеко расположены: 24 категория слилась с 3.

In [ ]:
#создаём словарь номер категории - категория
category_num=range(len(clusters))
category_name=clusters
category_dict=dict(zip(category_num,category_name))
#распространяем на исходную таблицу
labels=pd.DataFrame(columns=['num','name'])
labels['num']=model.labels_[n1:n2]
for i in range(len(labels)):
  labels['name'][i]=category_dict[labels['num'][i]]

result=text.join(labels,how='inner')

<ipython-input-16-fd7d1602c37b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['name'][i]=category_dict[labels['num'][i]]


In [ ]:
#вывод результатов
result.head()

,text,num,name
0,"❗️Восстановление аммиакопровода Тольятти — Одесса займет от 1 до 3 месяцев при наличии доступа к объекту, заявили в МИД РФ.\n\nВ ведомстве отметили, что Россия предпримет усилия для выяснения обст...",11,Политика
1,Дополнительные 39 миллионов рублей были выделены на социальные учреждения Марий Эл. Об этом заявили участники Съезда соцработников.\n\nСемь автобусов для детских учреждений будут закуплены на 29 м...,26,Финансы
2,«Россия уничтожена санкциями»\n Байден.\nАх-ха-ха:)))\n\nhttps://t.me/rt_russian/160646,3,Технологии
3,"❗️Правоохранители провели обыски в министерстве образования и науки Дагестана, сообщает ТАСС. \n\nВ правоохранительных органах уточнили, что обыски прошли по уголовному делу, связанному с выплатам...",14,Образование и познавательное
4,"👧👦 В загородных детских лагерях Марий Эл в данный момент отдыхают 2335 детей, и ещё 12265 ребят посещают 175 пришкольных лагерей.\n\nВсего на территории республики планируется открыть 200 оздорови...",9,Психология


## Модель обучения

In [ ]:
corpus=result['text']

#очистка текстов
for i in notebook.tqdm(range(len(corpus))):
  corpus[i]=clear_text(corpus[i])

 #лемматизация текстов
for i in notebook.tqdm(range(len(corpus))):
  corpus[i]=lemmatize(corpus[i])

  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-18-95945bbaa45b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus[i]=clear_text(corpus[i])


  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-18-95945bbaa45b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus[i]=lemmatize(corpus[i])


In [ ]:
features=corpus
target=result['num']

In [ ]:
#разделение выборок на тренировочную и тестовую
features_train,features_test,target_train,target_test=train_test_split(features,target,train_size=0.8)

In [ ]:
%%time
#обучение модели логистической регрессии с перебором гиперпараметров и взвешиванием классов
parametres={}
pipe_0=Pipeline([('tfidf',TfidfVectorizer(stop_words=list(stopwords))),
                 ('clf',LogisticRegression())])
best_model_0=GridSearchCV(estimator=pipe_0,
                          param_grid=parametres,
                          cv=8,
                          scoring=make_scorer(f1_score,average='micro'))
best_model_0.fit(features_train,target_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=8.
  warnings.warn(


CPU times: user 25.2 s, sys: 24.3 s, total: 49.5 s
Wall time: 32.6 s


GridSearchCV(cv=8,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['к', 'чуть',
                                                                    'свою',
                                                                    'всегда',
                                                                    'хорошо',
                                                                    'надо',
                                                                    'ей', 'ж',
                                                                    'иногда',
                                                                    'хоть',
                                                                    'ну', 'уже',
                                                                    'тот',
                                                                    'меня', 'о',
                                                                    'тогда',
                                                                    'да',
                                                                    'ничего',
                                                                    'из',
                                                                    'если',
                                                                    'про',
                                                                    'над',
                                                                    'как',
                                                                    'кто',
                                                                    'другой',
                                                                    'было',
                                                                    'чтобы',
                                                                    'ли', 'мой',
                                                                    'сейчас', ...])),
                                       ('clf', LogisticRegression())]),
             param_grid={}, scoring=make_scorer(f1_score, average=micro))

In [ ]:
#вывод результатов f1-меры для модели
print('f1-мера:',abs(best_model_0.best_score_))

f1-мера: 0.5075000000000001


In [ ]:
#предсказания на тестовой выборке
predictions_test=best_model_0.predict(features_test)
#вывод результатов f1-меры
print('f1-мера:',f1_score(predictions_test,target_test,average='micro'))

f1-мера: 0.515
